In [1]:
import torch
import torch.nn as nn
import math

In [2]:

# d_model = 512
# seq_len = 350

In [3]:
# This will convert input words into a 512 dimension input embedding
class InputEmbeddings(nn.Module):
    # d_model are embeddings dimension
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        # print(type(d_model))
        self.d_model = int(d_model)
        self.vocab_size = int(vocab_size)
        print(f"Vocab size: {vocab_size} and d_model: {d_model}")
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        return self.embedding(x.long()) * math.sqrt(self.d_model) #output = d_model

![image](https://aiml.com/wp-content/uploads/2023/09/example_of_positional_encoding_in_transformers.png)

In [4]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model: int, seq_len: int, dropout: float):
    super().__init__()
    self.d_model = d_model
    self.seq_len = seq_len
    self.dropout = nn.Dropout(dropout)

    # create a matrix of shape (seq_len, d_model)
    pe = torch.zeros(seq_len, d_model) # seq_len x d_model
    position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # seq_len x 1
    div_term = torch.exp(torch.arange(0, d_model, step=2).float() * (-math.log(10000) / d_model))
    # apply sin to even positions
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)

    # because we have batch of sentence
    # add a new position to pe
    pe = pe.unsqueeze(0) # (1, seq_len, d_model)
    self.register_buffer("pe", pe)  # save but not update the parameter

  def forward(self, x):
    # x -> (batch_size, seq_len, d_model)
    x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
    return  self.dropout(x)

![image](https://pic1.zhimg.com/v2-ea3d67c207c362e43875976c69f02db8_r.jpg)

In [5]:
# Formulae = x(i)- mean / sqrt(deviation**2 + epsilon)
class LayerNormalization(nn.Module):
  def __init__(self, eps:float = 10**-6):
    super().__init__()
    self.eps = eps
    self.alpha = nn.Parameter(torch.ones(1)) # multiplier
    self.bias = nn.Parameter(torch.zeros(1)) # bias

  def forward(self, x):
    mean = x.mean(dim = -1, keepdim=True)
    std = x.std(dim = -1, keepdim=True)
    return self.alpha * (x - mean) / (std + self.eps) + self.bias

In [6]:
class FeedForwardBlock(nn.Module):
  def __init__(self, d_model: int, d_ff: int, dropout: float):
    super().__init__()
    self.linear1 = nn.Linear(d_model, d_ff)
    self.dropout = nn.Dropout(dropout)
    self.linear2 = nn.Linear(d_ff, d_model)

  def forward(self, x):
    # (Batch, seq_len, d_model) --> (Batch, seq_len, d_ff) -> (Batch, seq_len, d_model)
    return self.linear2(self.dropout(torch.relu(self.linear1(x))))

![image](https://theaisummer.com/static/159ccc4fea2cef811b570d55cb594992/d2a60/multi-head-self-attention-block-diagram.png)

![image](https://www.ncbi.nlm.nih.gov/books/NBK597474/bin/515045_1_En_6_Fig2_HTML.jpg)

In [7]:
class MultiHeadAttentionBlock(nn.Module):
  def __init__(self,d_model, head: int, dropout: float):
    super().__init__()
    self.d_model = d_model
    self.head = head

    assert d_model % head == 0 , "d_model must be divisible by h"
    self.d_k = d_model // head

    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)

    self.w_o = nn.Linear(d_model, d_model)
    self.dropout = nn.Dropout(dropout)

  @staticmethod
  def attention(query, key, value, mask, dropout: nn.Dropout = None):
    d_k = query.shape[-1]
    # (Batch , head, seq_len, d_k) -> # (Batch , head, seq_len, seq_len)
    attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
      attention_scores.masked_fill_(mask == 0, -1e9)
    attention_scores = attention_scores.softmax(dim = -1) # (batch, head, seq_len, seq_len)
    if dropout is not None:
      attention_scores = dropout(attention_scores)
    return (attention_scores @ value), attention_scores

  def forward(self, q, k, v, mask):
    query = self.w_q(q) # (Batch, Seq_Len, d_model) --> (Batch, Seq_Len, d_model)
    key = self.w_k(k) # (Batch, Seq_Len, d_model) --> (Batch, Seq_Len, d_model)
    value = self.w_v(v) # (Batch, Seq_Len, d_model) --> (Batch, Seq_Len, d_model)

    query = query.view(query.shape[0], query.shape[1], self.head, self.d_k) # (Batch, Seq_Len, head, d_k)
    query = query.transpose(1, 2) # (Batch, head, Seq_Len, d_k)
    key = key.view(key.shape[0], key.shape[1], self.head, self.d_k) # (Batch, Seq_Len, head, d_k)
    key = key.transpose(1, 2) # (Batch, head, Seq_Len, d_k)
    value = value.view(value.shape[0], value.shape[1], self.head, self.d_k) # (Batch, Seq_Len, head, d_k)
    value = value.transpose(1, 2) # (Batch, head, Seq_Len, d_k)

    x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
    # (Batch, head, Seq_Len, d_k) --> (Batch, Seq_Len, head, d_k)
    x = x.transpose(1, 2).contiguous()
    # (Batch, Seq_Len, head, d_k) --> (Batch, Seq_Len, d_model
    x = x.view(x.shape[0], -1, self.head * self.d_k)
    # (Batch, Seq_Len, d_model) --> (Batch, Seq_Len, d_model)
    return self.w_o(x)

![image](https://lena-voita.github.io/resources/lectures/seq2seq/transformer/model-min.png)

In [8]:
class ResidualConnection(nn.Module):
  def __init__(self, dropout = float):
    super().__init__()
    self.dropout = nn.Dropout(dropout)
    self.norm = LayerNormalization()

  def forward(self, x, sublayer: nn.Module):
    return x + self.dropout(sublayer(self.norm(x)))

## Encoder block
![image](https://www.scaler.com/topics/images/encoder-block.webp)

In [9]:
class EncoderBlock(nn.Module):
  def __init__(self, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float):
    super().__init__()
    self.self_attention_block = self_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])

  def forward(self, x, src_mask):
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
    x = self.residual_connections[1](x, self.feed_forward_block)
    return x

In [10]:
class Encoder(nn.Module):
  def __init__(self, layers: nn.ModuleList) :
    super().__init__()
    self.layers = layers
    self.norm = LayerNormalization()

  def forward(self, x, src_mask):
    for layer in self.layers:
      x = layer(x, src_mask)
    return self.norm(x)

## Decoder

![image](https://lilianweng.github.io/posts/2018-06-24-attention/transformer-decoder.png)

In [11]:
class DecoderBlock(nn.Module):
  def __init__(self, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float):
    super().__init__()
    self.self_attention_block = self_attention_block
    self.cross_attention_block = cross_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)])

  def forward(self, x, encoder_output, src_mask, tgt_mask): # src_mask is for english and it is applied to encoder and tgt_mask will be applied to italian
    x = self.residual_connections[0](x, lambda x : self.self_attention_block(x, x, x, tgt_mask))
    x = self.residual_connections[1](x, lambda x : self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
    x = self.residual_connections[2](x, self.feed_forward_block)
    return x

In [12]:
class Decoder(nn.Module):
  def __init__(self, layers: nn.ModuleList):
    super().__init__()
    self.layers = layers
    self.norm = LayerNormalization()

  def forward(self, x, encoder_output, src_mask, tgt_mask):
    for layer in self.layers:
      x = layer(x, encoder_output, src_mask, tgt_mask)
    return self.norm(x)

In [13]:
# After decoder we have a block named linear , our projection layer does the work of that layer
# This layer will take (Batch, seq_len, d_model) and output (Batch, seq_len, vocab_size) ie take embedding and give word corresponding to that embedding from vocabulary
class ProjectionLayer(nn.Module):
  def __init__(self, d_model, vocab_size):
    super().__init__()
    self.proj = nn.Linear(d_model, vocab_size)

  def forward(self, x):
    # (Batch, seq_len, d_model) --> (Batch, seq_len, vocab_size)
    return torch.log_softmax(self.proj(x), dim = -1)

# Transformer Block

In [15]:
class Transformer(nn.Module):
  def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.tgt_embed = tgt_embed
    self.src_pos = src_pos
    self.tgt_pos = tgt_pos
    self.projection_layer = projection_layer

  def encode(self, src, src_mask):
    src = self.src_embed(src)
    src = self.src_pos(src)
    return self.encoder(src, src_mask)

  def decode(self, encoder_output, src_mask, tgt, tgt_mask):
    tgt = self.tgt_embed(tgt)
    tgt = self.tgt_pos(tgt)
    return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

  def project(self, x):
    return self.projection_layer(x)

## Method to build transformer

In [16]:
def build_transformer(src_vocab_size:int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model = 512, N = 6, heads = 8, dropout = 0.1, d_ff = 2048):
  # Create the embedding layers
  src_embed = InputEmbeddings(d_model, src_vocab_size)
  tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

  # Create the positional encoding layers
  src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
  tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

  # Create the encoder blocks
  encoder_blocks = []
  for _ in range(N):
    encoder_self_attention_block = MultiHeadAttentionBlock(d_model, heads, dropout)
    feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
    encoder_block = EncoderBlock(encoder_self_attention_block, feed_forward_block, dropout)
    encoder_blocks.append(encoder_block)

  # Create the decoder blocks
  decoder_blocks = []
  for _ in range(N):
    decoder_self_attention_block = MultiHeadAttentionBlock(d_model, heads, dropout)
    decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, heads, dropout)
    feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
    decoder_block = DecoderBlock(decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
    decoder_blocks.append(decoder_block)

  # Create the encoder and decoder
  encoder = Encoder(nn.ModuleList(encoder_blocks))
  decoder = Decoder(nn.ModuleList(decoder_blocks))

  # Create the projection layer
  projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

  # Create the transformer
  transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

  # Initialize the parameters
  for p in transformer.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform_(p)

  return transformer

# Preparing dataset

In [17]:
!pip install datasets
import datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [18]:
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

In [19]:
from pathlib import Path

In [20]:
def get_all_sentences(dataset, lang):
  for item in dataset:
    yield item['translation'][lang]

def get_or_build_tokenizer(config, ds, lang): # config , dataset, language
  # Given the language this method will build the tokenizer example below
  # config['tokenizer_file'] = '../tokenizers/tokenizer_{lang}.json
  tokenizer_path = Path(config["tokenizer_file"].format(lang))
  if not Path.exists(tokenizer_path):
    tokenizer = Tokenizer(WordLevel(unk_token = "[UNK]"))
    tokenizer.pre_tokenizer = Whitespace()
    # Unknown for word not in vocabulary , padding for matching seq length , start of sentance and end of sentance and finally all the words atleast occured twice will occur in our vocabulary
    trainer = WordLevelTrainer(special_tokens = ["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency = 2)
    tokenizer.train_from_iterator(get_all_sentences(ds,lang), trainer = trainer)
    tokenizer.save(str(tokenizer_path))
  else:
    tokenizer = Tokenizer.from_file(str(tokenizer_path))
  return tokenizer

In [21]:
from torch.utils.data import Dataset

def casual_mask(size):
  mask = torch.triu(torch.ones((1, size, size)), diagonal = 1).type(torch.int)
  return mask == 0

class BillingualDataset(Dataset):
  def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len): # src_lang => name of source language , tgt_lang => name of target language
    super().__init__()
    self.seq_len = seq_len
    self.ds = ds
    self.tokenizer_src = tokenizer_src
    self.tokenizer_tgt = tokenizer_tgt
    self.src_lang = src_lang
    self.tgt_lang = tgt_lang

    self.sos_token = torch.tensor([tokenizer_src.token_to_id("[SOS]")], dtype=torch.int64)
    self.eos_token = torch.tensor([tokenizer_src.token_to_id("[EOS]")], dtype=torch.int64)
    self.pad_token = torch.tensor([tokenizer_src.token_to_id("[PAD]")], dtype=torch.int64)

  def __len__(self):
    return len(self.ds)

  def __getitem__(self, idx):
    src_target_pair = self.ds[idx]
    src_text = src_target_pair['translation'][self.src_lang]
    tgt_text = src_target_pair['translation'][self.tgt_lang]

    # Transform the text into tokens
    enc_input_tokens = self.tokenizer_src.encode(src_text).ids
    dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids

    # Add sos, eos and padding to each sentence
    enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2  # -2 because we will add SOS and EOS
    dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1 # -1 because we will add SOS token , we don't add eos to input to decoder

    # Make sure the number of padding tokens is not negative. If it is, sentence is too long
    if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
      raise ValueError("Sentence is too long")

    # Add SOS and EOS token to the source text
    encoder_input = torch.cat(
        [
            self.sos_token,
            torch.tensor(enc_input_tokens, dtype=torch.int64),
            self.eos_token,
            torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype=torch.int64)
        ]
    )

    # Add only SOS token to the input to decoder
    decoder_input = torch.cat(
        [
            self.sos_token,
            torch.tensor(dec_input_tokens, dtype=torch.int64),
            torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype = torch.int64)
        ]
    )

    # Add only EOS token to the label or y (what we expect as output from the decoder)
    label = torch.cat(
        [
            torch.tensor(dec_input_tokens, dtype=torch.int64),
            self.eos_token,
            torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype = torch.int64)
        ]
    )

    assert encoder_input.size(0) == self.seq_len
    assert decoder_input.size(0) == self.seq_len
    assert label.size(0) == self.seq_len

    return {
        "encoder_input": encoder_input, # (seq_len)
        "decoder_input": decoder_input, # (seq_len
        "encoder_mask": (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0), # (1, 1, seq_len)
        "decoder_mask": (decoder_input != self.pad_token).unsqueeze(0).unsqueeze(0) & casual_mask(decoder_input.size(0)), #(1, Seq_len) & (1, Seq_len, Seq_len) # casual mask to prevent the attention to looking at the words occuring after
        "label": label, # (seq_len)
        "src_text": src_text,
        "tgt_text": tgt_text
    }

In [22]:
from torch.utils.data import random_split,DataLoader

def get_ds(config):
  ds_raw = load_dataset('opus_books', f'{config["lang_src"]}-{config["lang_tgt"]}', split="train")
  # build tokenizer
  tokenizer_src = get_or_build_tokenizer(config, ds_raw, config["lang_src"])
  tokenizer_tgt = get_or_build_tokenizer(config, ds_raw, config["lang_tgt"])

  # Keep 90% for training and 10% for validation
  train_ds_size = int(0.9 * len(ds_raw))
  train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, len(ds_raw) - train_ds_size])

  train_ds = BillingualDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, config["lang_src"], config["lang_tgt"], config["seq_len"])
  val_ds = BillingualDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, config["lang_src"], config["lang_tgt"], config["seq_len"])

  max_len_src = 0
  max_len_tgt = 0

  for item in ds_raw:
    src_ids = tokenizer_src.encode(item['translation'][config['lang_src']]).ids
    tgt_ids = tokenizer_tgt.encode(item['translation'][config['lang_tgt']]).ids
    max_len_src = max(max_len_src, len(src_ids))
    max_len_tgt = max(max_len_tgt, len(tgt_ids))

  print(f"Max length of source sentence {max_len_src}")
  print(f"Max length of target sentence {max_len_tgt}")

  train_dataloader = DataLoader(train_ds, batch_size=config["batch_size"], shuffle=True)
  val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True)

  return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt

# Training and validation


In [23]:
def get_model(config, vocab_src_len, vocab_tgt_len):
  model = build_transformer(vocab_src_len, vocab_tgt_len, config["seq_len"], config["seq_len"], d_model = config["d_model"])
  return model

In [24]:
def get_config():
  return {
      "batch_size": 8,
      "num_epochs": 20,
      "lr": 10**-3,
      "seq_len": 350,
      "d_model": 512,
      "lang_src": "en",
      "lang_tgt": "it",
      "model_folder":"weights",
      "model_filename":"tmodel_",
      "preload":None,
      "tokenizer_file":"tokenizer_{0}.json",
      "experiment_name":"runs/tmodel",
  }

def get_weights_file_path(config,epoch: str):
    model_folder = config['model_folder']
    model_basename = config['model_filename']
    model_filename = f"{model_basename}{epoch}.pt"
    return str( Path('.') / model_folder / model_filename )


## Validation


In [25]:
def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
  sos_idx = tokenizer_src.token_to_id("[SOS]")
  eos_idx = tokenizer_src.token_to_id("[EOS]")

  # Precompute the encoder output and reuse it for every token we get from the decoder
  encoder_output = model.encode(source, source_mask)

  # Initialize the decoder input with the sos token
  decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)
  while True:
    if decoder_input.size(1) == max_len:
      break

    # build mask for target
    decoder_mask = casual_mask(decoder_input.size(1)).type_as(source_mask).to(device)

    # Calculate the output of the decoder
    out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

    # Get the next token
    prob = model.project(out[:, -1])
    _, next_word = torch.max(prob, dim = 1)
    decoder_input = torch.cat([decoder_input, torch.empty(1,1).type_as(source).fill_(next_word.item()).to(device)], dim = 1)

    if next_word == eos_idx:
      break

  return decoder_input.squeeze(0)

def run_validation (model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, global_state, writer, num_examples = 2):
  model.eval()
  count = 0
  # source_texts = []
  # expected = []
  # predicted = []
  console_width = 80
  with torch.no_grad():
    for batch in validation_ds:
      count += 1
      encoder_input = batch["encoder_input"].to(device)
      encoder_mask = batch["encoder_mask"].to(device)

      assert encoder_input.size(0) == 1 , "Batch size must be 1 for validation"

      model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)

      source_text = batch['src_text'][0]
      target_text = batch['tgt_text'][0]
      model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())

      # source_texts.append(source_text)
      # expected.append(target_text)
      # predicted.append(model_out_text)

      # Print to console
      print_msg('-'*console_width)
      print_msg(f'SOURCE: {source_text}')
      print_msg(f'TARGET: {target_text}')
      print_msg(f'PREDICTED: {model_out_text}')

      if count == num_examples:
        print_msg('-'*console_width)
        break


## Training loop

In [26]:
# from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
def train_model(config):
  # Define the device
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"Using device: {device}")

  # Make sure the models folder exists
  Path(config["model_folder"]).mkdir(parents=True, exist_ok=True)

  train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
  model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

  # # Tensorboard
  # writer = SummaryWriter(config["experiment_name"])

  optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"], eps=1e-9)
  initial_epoch = 0
  global_step = 0

  if config["preload"]:
    model_filename = get_weights_file_path(config, config["preload"])
    print(f"Preloading model {model_filename}")
    state = torch.load("model_filename")
    initial_epoch = state["epoch"] + 1
    optimizer.load_state_dict(state["optimizer_state_dict"])
    global_step = state["global_step"]
    # model.load_state_dict(state["model_state_dict"])

  loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id("[PAD]"), label_smoothing=0.1).to(device)

  for epoch in range(initial_epoch, config["num_epochs"]):
    batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")
    for batch in batch_iterator:
      model.train()
      encoder_input = batch["encoder_input"].to(device) #(Batch, Seq_len)
      decoder_input = batch["decoder_input"].to(device) #(Batch, Seq_len)
      encoder_mask = batch["encoder_mask"].to(device) #(Batch, 1, 1, Seq_len)
      decoder_mask = batch["decoder_mask"].to(device) #(Batch, 1, Seq_len, Seq_len)

      # RUn the tensors through transformer
      encoder_output = model.encode(encoder_input, encoder_mask) # (Batch, Seq_len, d_model)
      decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
      proj_output = model.project(decoder_output) # (Batch, Seq_len, tgt_vocab_size)

      label = batch["label"].to(device) # (Batch, Seq_len)

      # Compute the loss
      loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))
      batch_iterator.set_postfix({"train_loss": f"{loss.item():6.3f}"})

      # Log the loss
      # writer.add_scalar("train_loss", loss.item(), global_step)
      # writer.flush()

      loss.backward()

      # Update the weights
      optimizer.step()
      optimizer.zero_grad()

      run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config["seq_len"], device, lambda msg: batch_iterator.write(msg), global_step, writer=None)
      model_filename = get_weights_file_path(config, f"{epoch:02d}_temp")
      torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "global_step": global_step
      }, model_filename)
      global_step += 1

    model_filename = get_weights_file_path(config, f"{epoch:02d}")
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "global_step": global_step
    }, model_filename)


In [ ]:
if __name__ == "__main__":
  config = get_config()
  train_model(config)

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

Max length of source sentence 309
Max length of target sentence 274
Vocab size: 15698 and d_model: 512
Vocab size: 22463 and d_model: 512


Processing Epoch 00:   0%|          | 0/3638 [00:05<?, ?it/s, train_loss=10.034]

...

--------------------------------------------------------------------------------
SOURCE: The good-natured Mary Evgenyevna shook with laughter at everything the amusing Prince said, and even Varenka, in a way new to Kitty, succumbed to the feeble but infectious laughter inspired by the Prince's jokes.
TARGET: La buona Mar’ja Evgenevna scoppiava a ridere a ogni facezia che diceva il principe, e perfino Varen’ka, cosa che Kitty non aveva notato mai, si sfiniva in un debole, ma contagioso riso suscitatole dagli scherzi del principe.
PREDICTED: , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , 

Processing Epoch 00:   1%|          | 39/3638 [07:53<12:16:37, 12.28s/it, train_loss=7.473]

--------------------------------------------------------------------------------
SOURCE: They agreed that an English plough ploughed better, that a scarifier worked quicker, but they found a thousand reasons why they could not use either the one or the other; and, though he was convinced that it would be necessary to lower his standards of farming, he disliked having to give up improvements the benefit of which was so clear.
TARGET: Essi convenivano che l’aratro arava meglio, che l’estirpatrice lavorava con maggiore rendimento, ma trovavano mille ragioni per non dover usare né l’uno né l’altra, e, sebbene egli fosse convinto che occorresse abbassare il livello dell’azienda, gli spiaceva rinunziare a perfezionamenti il cui vantaggio era tanto evidente.
PREDICTED: , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,

Processing Epoch 00:   1%|          | 39/3638 [07:57<12:16:37, 12.28s/it, train_loss=7.473]

--------------------------------------------------------------------------------
SOURCE: I understand conscription which touches the fate of my children, of my brothers and myself, and I am ready to discuss what concerns me; but how to dispose of forty thousand roubles of Zemstvo money, or how to try the idiot Aleshka, I neither understand nor can take part in.'
TARGET: Capisco il servizio militare perché interessa la sorte dei miei figli, dei miei fratelli e di me stesso; sono pronto a giudicare tutto quanto mi riguarda; ma giudicare se e come distribuire quarantamila rubli di denaro del distretto o giudicare Alëška lo scemo, io questo non lo capisco e non posso farlo.
PREDICTED: , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , 